In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os

import march_madness as mm
import random

In [2]:
def get_win_location_weight(loc):
    if loc == 'H':
        return 0.6
    elif loc == 'A':
        return 1.4
    else:
        return 1

def get_loss_location_weight(loc):
    if loc == 'A':
        return 1.4
    elif loc == 'H':
        return 0.6
    else:
        return 1

In [3]:
analysis = mm.Analysis()

In [4]:
analysis.load_season(2019)

In [5]:
analysis.calc_seasons_features()

2019


In [6]:
season = analysis.seasons[2019]

In [7]:
season.tourney_seeds

,Season,Seed,Numerical Seed
TeamID,,,
1181,2019,W01,1
1277,2019,W02,2
1261,2019,W03,3
1439,2019,W04,4
1280,2019,W05,5
...,...,...,...
1332,2019,Z12,12
1414,2019,Z13,13
1330,2019,Z14,14


In [8]:
team = season.teams[1280]

In [9]:
team.win_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
156236,2019,4,1280,95,1122,67,H,0
156340,2019,6,1280,77,1216,59,H,0
156528,2019,11,1280,79,1253,51,H,0
156807,2019,16,1280,61,1388,57,N,0
157024,2019,21,1280,88,1108,65,H,0
157164,2019,25,1280,65,1173,58,A,0
157353,2019,29,1280,90,1270,77,H,0
157496,2019,33,1280,82,1155,71,N,0
157656,2019,40,1280,70,1153,59,H,0
157828,2019,44,1280,98,1459,87,H,0


In [10]:
team.loss_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
156675,2019,14,1113,72,1280,67,N,0
158496,2019,64,1376,87,1280,82,H,0
158693,2019,68,1279,81,1280,77,A,0
159142,2019,78,1246,76,1280,55,H,0
159453,2019,85,1104,83,1280,79,H,0
159816,2019,93,1261,92,1280,88,A,0
159977,2019,96,1246,71,1280,67,A,0
160895,2019,117,1120,80,1280,75,H,0
161091,2019,120,1397,71,1280,54,H,0
161510,2019,130,1397,83,1280,76,N,0


In [11]:
season_data = pd.concat([team.win_data,team.loss_data]).sort_values('DayNum')

In [12]:
last10_data = season_data[-10:]
last10_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
160113,2019,99,1280,81,1104,62,H,0
160314,2019,103,1280,77,1116,67,A,0
160467,2019,107,1280,68,1208,67,A,0
160625,2019,110,1280,76,1376,61,H,0
160749,2019,113,1280,68,1281,49,H,0
160895,2019,117,1120,80,1280,75,H,0
161091,2019,120,1397,71,1280,54,H,0
161264,2019,124,1280,92,1401,81,H,0
161439,2019,129,1280,80,1401,54,N,0
161510,2019,130,1397,83,1280,76,N,0


In [13]:
last10_win_count = 0
last10_weighted_win_count = 0
last10_weighted_loss_count = 0

for i in last10_data.index:
    game = last10_data.loc[i]
    
    winner_id = game['WTeamID']
    winner_loc = game['WLoc']
    
    if winner_id == team.id:
        last10_win_count += 1
        last10_weighted_win_count += get_win_location_weight(winner_loc)
    else:
        last10_weighted_loss_count += get_loss_location_weight(winner_loc)
        
last10_win_pct = last10_win_count / last10_data.shape[0]
last10_weighted_win_pct = round(last10_weighted_win_count / (last10_weighted_win_count + last10_weighted_loss_count),3)

In [14]:
last10_win_pct

0.7

In [15]:
last10_weighted_win_pct

0.738

In [16]:
last5_data = season_data[-5:]
last5_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
160895,2019,117,1120,80,1280,75,H,0
161091,2019,120,1397,71,1280,54,H,0
161264,2019,124,1280,92,1401,81,H,0
161439,2019,129,1280,80,1401,54,N,0
161510,2019,130,1397,83,1280,76,N,0


In [17]:
last5_win_count = 0
last5_weighted_win_count = 0
last5_weighted_loss_count = 0

for i in last5_data.index:
    game = last5_data.loc[i]
    
    winner_id = game['WTeamID']
    winner_loc = game['WLoc']
    
    if winner_id == team.id:
        last5_win_count += 1
        last5_weighted_win_count += get_win_location_weight(winner_loc)
    else:
        last5_weighted_loss_count += get_loss_location_weight(winner_loc)
        
last5_win_pct = last5_win_count / last5_data.shape[0]
last5_weighted_win_pct = round(last5_weighted_win_count / (last5_weighted_win_count + last5_weighted_loss_count),3)

In [18]:
last5_win_pct

0.4

In [19]:
last5_weighted_win_pct

0.421

In [20]:
team.id

1280

In [21]:
last10_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
160113,2019,99,1280,81,1104,62,H,0
160314,2019,103,1280,77,1116,67,A,0
160467,2019,107,1280,68,1208,67,A,0
160625,2019,110,1280,76,1376,61,H,0
160749,2019,113,1280,68,1281,49,H,0
160895,2019,117,1120,80,1280,75,H,0
161091,2019,120,1397,71,1280,54,H,0
161264,2019,124,1280,92,1401,81,H,0
161439,2019,129,1280,80,1401,54,N,0
161510,2019,130,1397,83,1280,76,N,0


In [22]:
conf_tourney_wins = 0

for i in last10_data.index[::-1]:
    game = last10_data.loc[i]
    
    loc = game['WLoc']
    
    if loc != 'N':
        break
    else:
        if game['WTeamID'] == team.id:
            conf_tourney_wins += 1

In [23]:
conf_tourney_wins

1

In [24]:
last_game_index = last5_data.index[-1]

In [25]:
last_game_index

161510

In [26]:
last_game = last5_data.loc[last_game_index]

In [27]:
if last_game['WTeamID'] == team.id:
    conf_champ = 1
else:
    conf_champ = 0

In [28]:
conf_champ

0